# Demo Resource Synchronization
Synchronization of resources means that before objects can be moved from one location to another, the processor as well as the origin and the destination resource must be available. If a resource is not available the process will wait untill the resource available.

Currently this waiting time is not recorded in the logs. The waiting time can be derived from the activity log by identifying time differences between the completion of an activity and the start of the subsequent activity.

This is an incomplete feature since preserve reservations can not be released automatically. Recommendation is to have a request_resource and a release_resource activity.

The current mechanism is based on two attributes of activities
- **keep_resources** a list of resources, which should not be released at the end of an activity
- **reserved_resources** is a dictionary, should be initialized for all activities of a process with the same variable. This will enable the transfer of reserved resources from one activity to another one.

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

my_env.registry = {}

## Definition of Site

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5d",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 10,  # The capacity of the site
    "level": 8,
}  # The actual volume of the site

# The two objects used for the simulation
from_site = Site(**data_from_site)

init
level: 8
completed init


## Definition of Vessels

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10


# TSHD variables
data_hopper1 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 4,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}

hopper1 = TransportProcessingResource(**data_hopper1)

data_hopper2 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 02",  # Name
    "ID": "5dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 4,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
hopper2 = TransportProcessingResource(**data_hopper2)

init
level: 0
completed init
init
level: 0
completed init


## Definition of the process for hopper1
Then **requested_resources1** variable is used to initialize all activities below. This enables the transfer of reserved resources to the next activity.
The **keep_resources** parameter tells that after hopper1 has received an object from *from_site* it should not release the *from_site* resource.

In [4]:
requested_resources1 = {}
shift_amount_activity_loading_data1 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer1",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper1,
    "origin": from_site,
    "destination": hopper1,
    "amount": 1,
    "duration": 20,
    "postpone_start": True,
#     "keep_resources":[from_site],
#     "requested_resources": requested_resources1,
}
activity1 = model.ShiftAmountActivity(**shift_amount_activity_loading_data1)

sequential_activity_data1 = {
    "env": my_env,
    "name": "Sequential process1",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": [activity1],
    "postpone_start": True,
#     "requested_resources": requested_resources1,
}
seq_activity1 = model.SequentialActivity(**sequential_activity_data1)

while_data1 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while1",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_process": seq_activity1,
    "condition_event": [{"or":[{"type":"container", "concept": hopper1, "state":"full"},
               {"type":"container", "concept": from_site, "state":"empty"}]
         }],
    "postpone_start": False,
#     "requested_resources": requested_resources1,
}
while_activity1 = model.WhileActivity(**while_data1)

while Activity keep_resources []
get_full_event : default
start get_available
start put_available
register new event 
evaluate event <Event() object at 0x198e0e102c8> as triggered False and processed False
evaluate event <Event() object at 0x198e0e10348> as triggered False and processed False
start event instance None


## Definition of process for hopper2
This is a normal process without special parameters.

In [5]:
shift_amount_activity_loading_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer2",  # We are moving soil
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper2,
    "origin": from_site,
    "destination": hopper2,
    "amount": 1,
    "duration": 20,
    "postpone_start": True,
}
activity2 = model.ShiftAmountActivity(**shift_amount_activity_loading_data2)

sequential_activity_data2 = {
    "env": my_env,
    "name": "Sequential process2",
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": [activity2],
    "postpone_start": True,
}
seq_activity2 = model.SequentialActivity(**sequential_activity_data2)

while_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while2",  # We are moving soil
    "ID": "5dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_process": seq_activity2,
    "condition_event": [{"or":[{"type":"container", "concept": hopper2, "state":"full"},
               {"type":"container", "concept": from_site, "state":"empty"}]
         }],
    "postpone_start": False,
}
while_activity2 = model.WhileActivity(**while_data2)


while Activity keep_resources []
get_full_event : default
start get_available
start put_available
check put_available events
register new event 
evaluate event <Event() object at 0x198e0e20708> as triggered False and processed False
evaluate event <Event() object at 0x198e0e20788> as triggered False and processed False
start event instance None


In [6]:
my_env.run()


conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5d': 8, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 4}
conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {<simpy.resources.resource.Resource object at 0x00000198E0E062C8>: <Request() object at 0x198e0e046c8>}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5d': 8, 'destination.5dbbbdf6-4589-11e9-95a2-b469212bff5b': 4}
put_callback - id_ default
{}
put_callback - id_ default
{'default': {4: <Event() object at 0x198e0e102c8>}}
amount :4
put_callback - id_ default
{'default': {4: <Event() object at 0x198e0e20708>}}
amount :4
destination request : {<simpy.resources.resource.Resource object at 0x00000198E0E062C8>: <Request() object at 0x198e0e046c8>, <simpy.resources.resource.Resource object at 0x00000198E0E067C8>: <Request() object a

In [7]:
log_df = pd.DataFrame(from_site.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data_rep = data.drop_duplicates()
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,transfer default to Hopper 02,START,1970-01-01 01:00:00,1,5dbbbdf7-4589-11e9-bf3b-b469212bff52
1,transfer default to Hopper 01,START,1970-01-01 01:00:00,1,6dbbbdf7-4589-11e9-bf3b-b469212bff52
2,transfer default to Hopper 02,STOP,1970-01-01 01:00:20,1,5dbbbdf7-4589-11e9-bf3b-b469212bff52
3,transfer default to Hopper 01,STOP,1970-01-01 01:00:20,1,6dbbbdf7-4589-11e9-bf3b-b469212bff52
4,transfer default to Hopper 01,START,1970-01-01 01:00:20,1,6dbbbdf7-4589-11e9-bf3b-b469212bff52
5,transfer default to Hopper 02,START,1970-01-01 01:00:20,1,5dbbbdf7-4589-11e9-bf3b-b469212bff52
6,transfer default to Hopper 01,STOP,1970-01-01 01:00:40,1,6dbbbdf7-4589-11e9-bf3b-b469212bff52
7,transfer default to Hopper 02,STOP,1970-01-01 01:00:40,1,5dbbbdf7-4589-11e9-bf3b-b469212bff52
8,transfer default to Hopper 02,START,1970-01-01 01:00:40,1,5dbbbdf7-4589-11e9-bf3b-b469212bff52
9,transfer default to Hopper 01,START,1970-01-01 01:00:40,1,6dbbbdf7-4589-11e9-bf3b-b469212bff52


In [8]:
pd.DataFrame(from_site.log).head(50)

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,transfer default to Hopper 02,1970-01-01 01:00:00,1,POINT (4.18055556 52.18664444),5dbbbdf7-4589-11e9-bf3b-b469212bff52,START
1,transfer default to Hopper 01,1970-01-01 01:00:00,1,POINT (4.18055556 52.18664444),6dbbbdf7-4589-11e9-bf3b-b469212bff52,START
2,transfer default to Hopper 02,1970-01-01 01:00:20,1,POINT (4.18055556 52.18664444),5dbbbdf7-4589-11e9-bf3b-b469212bff52,STOP
3,transfer default to Hopper 01,1970-01-01 01:00:20,1,POINT (4.18055556 52.18664444),6dbbbdf7-4589-11e9-bf3b-b469212bff52,STOP
4,transfer default to Hopper 01,1970-01-01 01:00:20,1,POINT (4.18055556 52.18664444),6dbbbdf7-4589-11e9-bf3b-b469212bff52,START
5,transfer default to Hopper 02,1970-01-01 01:00:20,1,POINT (4.18055556 52.18664444),5dbbbdf7-4589-11e9-bf3b-b469212bff52,START
6,transfer default to Hopper 01,1970-01-01 01:00:40,1,POINT (4.18055556 52.18664444),6dbbbdf7-4589-11e9-bf3b-b469212bff52,STOP
7,transfer default to Hopper 02,1970-01-01 01:00:40,1,POINT (4.18055556 52.18664444),5dbbbdf7-4589-11e9-bf3b-b469212bff52,STOP
8,transfer default to Hopper 02,1970-01-01 01:00:40,1,POINT (4.18055556 52.18664444),5dbbbdf7-4589-11e9-bf3b-b469212bff52,START
9,transfer default to Hopper 01,1970-01-01 01:00:40,1,POINT (4.18055556 52.18664444),6dbbbdf7-4589-11e9-bf3b-b469212bff52,START


Interesting in this scenario is that the process of hopper2 is not executed at all because hopper1 takes requests the resource *from_site* and never releases it again, thus hopper2 process can not start.

By commenting out the **keep_resource** parameter in shift amount activity of hopper1, will result in hopper1 and hopper2 getting access to the resource *from_site* one after each other.